In [1]:
import FiinQuantX as fg
from FiinQuantX import FiinSession

In [2]:
# Đăng nhập vào FiinQuant
try:
    username = 'DSTC_20@fiinquant.vn'
    password = 'Fiinquant0606'
    
    print("Đang đăng nhập vào FiinQuant...")
    client = FiinSession(
        username=username,
        password=password,
    ).login()
    
    print("✓ Đăng nhập thành công!")
    print(f"Client object: {type(client)}")
    
except Exception as e:
    print(f"❌ Lỗi đăng nhập: {e}")
    print("Kiểm tra lại username/password hoặc kết nối internet")

Đang đăng nhập vào FiinQuant...
✓ Đăng nhập thành công!
Client object: <class 'FiinQuantX.FiinQuant.FiinSession'>


In [ ]:
# LẤY TẤT CẢ DỮ LIỆU CỔ PHIẾU 3 NĂM (FULL FIELDS)
import pandas as pd
from datetime import datetime, timedelta
import time

# Đọc danh sách ticker đã hợp lệ
valid_tickers_df = pd.read_csv("all_tickers.csv")
valid_tickers = valid_tickers_df["Ticker"].tolist()
print(f"Sẽ lấy dữ liệu FULL cho {len(valid_tickers)} ticker hợp lệ")

# Lấy tất cả các trường dữ liệu
ALL_FIELDS = ['open', 'high', 'low', 'close', 'volume', 'bu', 'sd', 'fb', 'fs', 'fn']
print(f"📊 Các trường: {ALL_FIELDS}")

# Lấy dữ liệu với batch
BATCH_SIZE = 30  # Giảm batch size vì nhiều field hơn
all_data = []
batches = [valid_tickers[i:i+BATCH_SIZE] for i in range(0, len(valid_tickers), BATCH_SIZE)]

print(f"Xử lý {len(batches)} batch...")
start_time = time.time()

for i, batch in enumerate(batches):
    print(f"Batch {i+1}/{len(batches)}: ", end="", flush=True)
    
    try:
        # Lấy dữ liệu với TẤT CẢ các trường
        data = client.Fetch_Trading_Data(
            realtime=True,
            tickers=batch,
            fields=ALL_FIELDS,  # LẤY TẤT CẢ
            adjusted=True,
            by="1d",
            from_date=(datetime.now() - timedelta(days=1095)).strftime("%Y-%m-%d")  # 3 năm dữ liệu
        ).get_data()
        
        if not data.empty:
            all_data.append(data)
            print(f"✅ {len(data)} rows, {len(data.columns)} cols")
            
            # In cột lần đầu
            if i == 0:
                print(f"\n📋 Columns: {list(data.columns)}")
        else:
            print("❌ Empty")
            
    except Exception as e:
        print(f"❌ ERROR: {str(e)[:40]}...")
    
    time.sleep(0.1)
    
    # Progress mỗi 10 batch
    if (i + 1) % 10 == 0:
        elapsed = time.time() - start_time
        total_rows = sum(len(df) for df in all_data)
        print(f"\n--- Progress: {i+1}/{len(batches)}, {total_rows:,} rows, {elapsed:.1f}s ---")

# Gộp tất cả dữ liệu
if all_data:
    print(f"\n🔄 Gộp {len(all_data)} batch...")
    final_data = pd.concat(all_data, ignore_index=True)
    
    # Sắp xếp theo ticker và ngày
    final_data = final_data.sort_values(['ticker', 'timestamp'])
    
    print(f"\n📊 THỐNG KÊ:")
    print(f"📏 Rows: {len(final_data):,}")
    print(f"📋 Columns: {len(final_data.columns)} - {list(final_data.columns)}")
    print(f"🏢 Tickers: {final_data['ticker'].nunique()}")
    print(f"📅 Từ: {final_data['timestamp'].min()}")
    print(f"📅 Đến: {final_data['timestamp'].max()}")
    
    # Lưu file
    output_file = f"stock_full_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    final_data.to_csv(output_file, index=False)
    print(f"\n💾 ĐÃ LƯU: {output_file}")
    
    # Hiển thị mẫu dữ liệu
    print(f"\n📋 MẪU DỮ LIỆU:")
    pd.set_option('display.max_columns', None)
    print(final_data.head())
    
    # Top 5 ticker
    print(f"\n🏆 TOP 5 TICKER:")
    top_tickers = final_data['ticker'].value_counts().head()
    for ticker, count in top_tickers.items():
        sample = final_data[final_data['ticker'] == ticker].iloc[-1]
        print(f"  {ticker}: {count} ngày - Close: {sample.get('close', 'N/A'):,}")
        
else:
    print("❌ Không có dữ liệu nào!")

elapsed = time.time() - start_time
print(f"\n🎉 HOÀN THÀNH trong {elapsed:.1f}s ({elapsed/60:.1f} phút)")


Sẽ lấy dữ liệu FULL cho 1606 ticker hợp lệ
📊 Các trường: ['open', 'high', 'low', 'close', 'volume', 'bu', 'sd', 'fb', 'fs', 'fn']
Xử lý 54 batch...
Batch 1/54: Fetching data, it may take a while. Please wait...
✅ 18508 rows, 12 cols

📋 Columns: ['ticker', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'bu', 'sd', 'fb', 'fs', 'fn']
Batch 2/54: Fetching data, it may take a while. Please wait...
✅ 22454 rows, 12 cols
Batch 3/54: Fetching data, it may take a while. Please wait...
✅ 22470 rows, 12 cols
Batch 4/54: Fetching data, it may take a while. Please wait...
✅ 22007 rows, 12 cols
Batch 5/54: Fetching data, it may take a while. Please wait...
✅ 22470 rows, 12 cols
Batch 6/54: Fetching data, it may take a while. Please wait...
✅ 22442 rows, 12 cols
Batch 7/54: Fetching data, it may take a while. Please wait...
✅ 22463 rows, 12 cols
Batch 8/54: Fetching data, it may take a while. Please wait...
✅ 22457 rows, 12 cols
Batch 9/54: Fetching data, it may take a while. Please wait...
✅